In [1]:
import numpy as np
import pandas as pd
import re
from PyPDF2 import PdfFileReader

# Load the PDF File

In [2]:
# open the PDF file
pdfFile = open('/Users/johnathonsmith/Downloads/293RDCV.PEND.CASE_1642601189564_-1416322429.pdf', 'rb')


# Read the File

In [3]:
# create PDFFileReader object to read the file
pdfReader = PdfFileReader(pdfFile)

# Check For Encryption

In [4]:
if pdfReader.isEncrypted == True:
    print("PDF File is encrypted. Password is needed to continue.")
else:
    print("PDF File is not encrypted. Password is not needed.")

PDF File is encrypted. Password is needed to continue.


# Print Document Info

In [5]:
#print("PDF File name: " + str(pdfReader.getDocumentInfo().title))
#print("PDF File created by: " + str(pdfReader.getDocumentInfo().creator))
#print("- - - - - - - - - - - - - - - - - - - -")

# Try Using PDFminer3

In [6]:
#imports
import pandas as pd

from pdfminer3.layout import LAParams, LTTextBox
from pdfminer3.pdfpage import PDFPage
from pdfminer3.pdfinterp import PDFResourceManager
from pdfminer3.pdfinterp import PDFPageInterpreter
from pdfminer3.converter import PDFPageAggregator
from pdfminer3.converter import TextConverter
import io

In [7]:
#Set up resource manager to handle pdf content. text, images, etc.
resource_manager = PDFResourceManager()

#Used to display text
fake_file_handle = io.StringIO()

#Set up converter
converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())

#Set up page interpreter
page_interpreter = PDFPageInterpreter(resource_manager, converter)

with open('/Users/johnathonsmith/Downloads/293RDCV.PEND.CASE_1642601189564_-1416322429.pdf', 'rb') as fh:

    for page_number, page in enumerate(PDFPage.get_pages(fh,
                                  caching=True,
                                  maxpages = 14,
                                  check_extractable=True)):
        #Only read page 1 for now (index = 0)
        if page_number == 0:
            page_interpreter.process_page(page)

    text = fake_file_handle.getvalue()

# close open handles
converter.close()
fake_file_handle.close()

print(text)

                COUNTY OF MAVERICK 293RD DISTRICT COURT CIVIL PENDING CASES AS OF 01/19/2022

                                           RAN ON JANUARY 19, 2022 09:02am PAGE: 1

        CAUSE #             FILE DATE CAUSE OF ACTION           DOCKT DATE  DOCKET TYPE       ANS FILED  CR CASE #

PLAINTIFF NAME                PLAINTIFF ATTORNEY        DEFENDANT NAME                DEFENDANT ATTORNEY

        7815-OTH           02/22/1985 1                                                                  

        ATTORNEY GENERAL              MARIA LARSON              MENDEZ, CARLOS GILBERTO                            
        MENDEZ, OLGA LIDIA                                                                                         
        -------------------------------------------------------------------------------------------------------------
        90-11-10256-CV     11/08/1990 1                                                                  

        ATTORNEY GENERAL            

In [8]:
#Now try finding places to break it down a bit
print(repr(text))

'                COUNTY OF MAVERICK 293RD DISTRICT COURT CIVIL PENDING CASES AS OF 01/19/2022\n\n                                           RAN ON JANUARY 19, 2022 09:02am PAGE: 1\n\n        CAUSE #             FILE DATE CAUSE OF ACTION           DOCKT DATE  DOCKET TYPE       ANS FILED  CR CASE #\n\nPLAINTIFF NAME                PLAINTIFF ATTORNEY        DEFENDANT NAME                DEFENDANT ATTORNEY\n\n        7815-OTH           02/22/1985 1                                                                  \n\n        ATTORNEY GENERAL              MARIA LARSON              MENDEZ, CARLOS GILBERTO                            \n        MENDEZ, OLGA LIDIA                                                                                         \n        -------------------------------------------------------------------------------------------------------------\n        90-11-10256-CV     11/08/1990 1                                                                  \n\n        ATTORNEY GEN

In [9]:
#First, strip leading and trailing whitespaces
text = text.strip()
print(repr(text))

'COUNTY OF MAVERICK 293RD DISTRICT COURT CIVIL PENDING CASES AS OF 01/19/2022\n\n                                           RAN ON JANUARY 19, 2022 09:02am PAGE: 1\n\n        CAUSE #             FILE DATE CAUSE OF ACTION           DOCKT DATE  DOCKET TYPE       ANS FILED  CR CASE #\n\nPLAINTIFF NAME                PLAINTIFF ATTORNEY        DEFENDANT NAME                DEFENDANT ATTORNEY\n\n        7815-OTH           02/22/1985 1                                                                  \n\n        ATTORNEY GENERAL              MARIA LARSON              MENDEZ, CARLOS GILBERTO                            \n        MENDEZ, OLGA LIDIA                                                                                         \n        -------------------------------------------------------------------------------------------------------------\n        90-11-10256-CV     11/08/1990 1                                                                  \n\n        ATTORNEY GENERAL            

In [10]:
#Now separate header and body text info

#For header
header = text[:392]
print(repr(header))

'COUNTY OF MAVERICK 293RD DISTRICT COURT CIVIL PENDING CASES AS OF 01/19/2022\n\n                                           RAN ON JANUARY 19, 2022 09:02am PAGE: 1\n\n        CAUSE #             FILE DATE CAUSE OF ACTION           DOCKT DATE  DOCKET TYPE       ANS FILED  CR CASE #\n\nPLAINTIFF NAME                PLAINTIFF ATTORNEY        DEFENDANT NAME                DEFENDANT ATTORNEY\n\n        '


In [11]:
#For body
text = text[392:]
print(repr(text))

'7815-OTH           02/22/1985 1                                                                  \n\n        ATTORNEY GENERAL              MARIA LARSON              MENDEZ, CARLOS GILBERTO                            \n        MENDEZ, OLGA LIDIA                                                                                         \n        -------------------------------------------------------------------------------------------------------------\n        90-11-10256-CV     11/08/1990 1                                                                  \n\n        ATTORNEY GENERAL              ROLANDO MENCHACA          MARTINEZ, HECTOR RENE                              \n        MARTINEZ, ROSALINDA                                                                                        \n        -------------------------------------------------------------------------------------------------------------\n        90-12-10326-CV     12/14/1990 INJURY OR DAMAGE OTHER T                     

In [12]:
#Now split each case by the dashed lines
cases = text.split('-------------------------------------------------------------------------------------------------------------')
cases

['7815-OTH           02/22/1985 1                                                                  \n\n        ATTORNEY GENERAL              MARIA LARSON              MENDEZ, CARLOS GILBERTO                            \n        MENDEZ, OLGA LIDIA                                                                                         \n        ',
 '\n        90-11-10256-CV     11/08/1990 1                                                                  \n\n        ATTORNEY GENERAL              ROLANDO MENCHACA          MARTINEZ, HECTOR RENE                              \n        MARTINEZ, ROSALINDA                                                                                        \n        ',
 '\n        90-12-10326-CV     12/14/1990 INJURY OR DAMAGE OTHER T                                           \n\n        DIAZ CARPENTER, ENRIQUETA     JAY T. KIMBROUGH          BOWLES JR., RUDY              RICHARD TINSMAN      \n        ',
 '\n        91-06-10598-CV     10/22/1993 MOTION ENFO

In [13]:
len(cases)

8

In [14]:
cases[0]

'7815-OTH           02/22/1985 1                                                                  \n\n        ATTORNEY GENERAL              MARIA LARSON              MENDEZ, CARLOS GILBERTO                            \n        MENDEZ, OLGA LIDIA                                                                                         \n        '

In [15]:
#Only work with one case for now
test_case = cases[1]
print(repr(test_case))

'\n        90-11-10256-CV     11/08/1990 1                                                                  \n\n        ATTORNEY GENERAL              ROLANDO MENCHACA          MARTINEZ, HECTOR RENE                              \n        MARTINEZ, ROSALINDA                                                                                        \n        '


In [16]:
#Strip the leading white space
test_case = test_case.lstrip()
print(repr(test_case))

'90-11-10256-CV     11/08/1990 1                                                                  \n\n        ATTORNEY GENERAL              ROLANDO MENCHACA          MARTINEZ, HECTOR RENE                              \n        MARTINEZ, ROSALINDA                                                                                        \n        '


In [17]:
#Try splitting on the new lines
case_info = test_case.split('\n')

In [18]:
len(case_info)

5

In [19]:
#I can isolate the cause number, file date, cause of action, docket date, docket type, 
#ans filed date, and cr number from here
case_info[0]

'90-11-10256-CV     11/08/1990 1                                                                  '

In [20]:
#Date starts 19 characters in.
case_info[0][19:]

'11/08/1990 1                                                                  '

In [21]:
#Cause of action start 30 characters in
case_info[0][30:]

'1                                                                  '

In [22]:
#Docket date starts 56 characters in
case_info[0][56:]

'                                         '

In [23]:
#Docket type starts 68 characters in
case_info[0][68:]

'                             '

In [24]:
#ANS Filed date starts 86 characters in
case_info[0][86:]

'           '

In [25]:
#CR number starts after 96 characters on the first line
case_info[0][96:]

' '

In [26]:
#Appears to be an extra new line
case_info[1]

''

In [27]:
#The first line of names starts 8 characters in. This should be the same for each case
case_info[2][8:]

'ATTORNEY GENERAL              ROLANDO MENCHACA          MARTINEZ, HECTOR RENE                              '

__Name Spacing__

The names appear to all start at the same place every time. Here is what I have found:

* Plaintiff Name: 8 Characters in
* Plaintiff Attorney: 38 Characters in
* Defendant Name: 64 Characters in
* Defendant Attorney: 94 Characters in

These numbers seem consistent regardless of what line they appear on.

In [28]:
#The second name starts 38 characters in. I hope this is the same for all of them
case_info[2][38:]

'ROLANDO MENCHACA          MARTINEZ, HECTOR RENE                              '

In [29]:
#The third name starts 64 characters in. I hope this is also the same for all of them
case_info[2][64:]

'MARTINEZ, HECTOR RENE                              '

In [30]:
#The fourth name starts 94 characters in
case_info[2][94:]

'                     '

In [31]:
case_info[3]

'        MARTINEZ, ROSALINDA                                                                                        '

In [32]:
#The second plaintiff name starts 8 characters in. This should be the same for each case
case_info[3][8:]

'MARTINEZ, ROSALINDA                                                                                        '

In [33]:
#The second defendant name starts 64 characters in.
case_info[3][64:]

'                                                   '

# Start Isolating Info

Isolate the case info and add it to a temp dict. When complete the temp dict will be appended to the container dictionary. Finally, when the info for each case has been extracted, convert the container dict to a dataframe.

In [34]:
#Create a container dict to store all case info
container_dict = {}

#Create a temp dict that stores individual case info
case_dict = {}

__County__

The county name will be found in the header info of each page. The easiest way I can think to isolate the county is to just use an if statement and check for each county name in the header text. There should only be one match, so it should be fine. Possible counties: Maverick, Dimmit, Zavala

In [35]:
header

'COUNTY OF MAVERICK 293RD DISTRICT COURT CIVIL PENDING CASES AS OF 01/19/2022\n\n                                           RAN ON JANUARY 19, 2022 09:02am PAGE: 1\n\n        CAUSE #             FILE DATE CAUSE OF ACTION           DOCKT DATE  DOCKET TYPE       ANS FILED  CR CASE #\n\nPLAINTIFF NAME                PLAINTIFF ATTORNEY        DEFENDANT NAME                DEFENDANT ATTORNEY\n\n        '

In [36]:
#Use if statement to check for county names inside the header info
if header.count('MAVERICK') >= 1:
    county = 'Maverick'
elif header.count('DIMMIT') >= 1:
    county = 'Dimmit'
elif header.count('ZAVALA') >= 1:
    county = 'Zavala'
else:
    county = 'Something went wrong!'
    
county

'Maverick'

In [37]:
#Add to dict
case_dict['County'] = county

__Cause Number__

In [38]:
#Get the cause number
cause_num = case_info[0][:19].strip()
cause_num

'90-11-10256-CV'

In [39]:
#Add to dict
case_dict['Cause Number'] = cause_num

__File Date__

In [40]:
file_date = case_info[0][15:30].strip()
file_date

'11/08/1990'

In [41]:
#Add to dict
case_dict['File Date'] = file_date

__Cause of Action__

In [42]:
coa = case_info[0][30:56].strip()
coa

'1'

In [43]:
#Add to dict
case_dict['Cause of Action'] = coa

__Docket Date__

In [44]:
docket_date = case_info[0][56:68].strip()
docket_date

''

In [45]:
#Add to dict
case_dict['Docket Date'] = docket_date

__Docket Type__

In [46]:
docket_type = case_info[0][68:86].strip()
docket_type

''

In [47]:
#Add to dict
case_dict['Docket Type'] = docket_type

__ANS File Date__

In [48]:
ans_date = case_info[0][86:].strip()
ans_date

''

In [49]:
#Add to dict
case_dict['ANS Filed'] = ans_date

__CR Number__

In [50]:
cr_num = case_info[0][96:].strip()
cr_num

''

In [51]:
#Add to dict
case_dict['CR Number'] = cr_num

__Names__

The next few entries will be tricky. If there is more than one name associated with each label (plaintiff name, plaintiff attorney, defendant name, defendant attorney), then there will be more than one line of names. In my function, I'll have to set up a loop that can accurately pick up all of the names. For now, I'll make each label a list of names. If it needs to be changed to a string of names later, I can do that too. I'll also have to build a function that checks to see if each segment is composed entirely of whitespace. That way, if it is, we know there isn't a name for that particular label. 

In [52]:
#Will need to set up the lists before the loop. They will be added to the dict after the loop
plaintiff_names = []
plaintiff_attorneys = []
defendant_names = []
defendant_attorneys = []

#The names always start on the third line (index = 2)
for i in range(2, len(case_info)):
    #Get the plaintiff name on current line
    plaintiff_name = case_info[i][:38]
    
    #Check if plaintiff_name is all whitesapace. If not, strip it and add to names list
    #Also check that the string is not empty
    if plaintiff_name.isspace() == False and len(plaintiff_name) > 0:
        plaintiff_names.append(plaintiff_name.strip())
        
    #Get the plaintiff attorney on current line
    plaintiff_attorney = case_info[i][38:64]
    
    #Check if plaintiff_attorney is all whitespace. If not, strip it and add to list
    #Also check that the string is not empty
    if plaintiff_attorney.isspace() == False and len(plaintiff_attorney) > 0:
        plaintiff_attorneys.append(plaintiff_attorney.strip())
        
    #Get the defendant name on current line
    defendant_name = case_info[i][64:94]
    
    #Check if defendant_name is all whitespace. If not, strip it and add to list
    #Also check that the string is not empty
    if defendant_name.isspace() == False and len(defendant_name) > 0:
        defendant_names.append(defendant_name.strip())
    
    #Get the defendant attorney on current line
    defendant_attorney = case_info[i][94:]
    
    #Check if the defendant_attorney is all whitespace. If not, strip it and add to list
    #Also check that the string is not empty
    if defendant_attorney.isspace() == False and len(defendant_attorney) > 0:
        defendant_attorneys.append(defendant_attorney.strip())

In [53]:
#Now add all names to the dict
case_dict['Plaintiff Name'] = plaintiff_names
case_dict['Plaintiff Attorney'] = plaintiff_attorneys
case_dict['Defendant Name'] = defendant_names
case_dict['Defendant Attorney'] = defendant_attorneys

In [54]:
case_dict

{'County': 'Maverick',
 'Cause Number': '90-11-10256-CV',
 'File Date': '11/08/1990',
 'Cause of Action': '1',
 'Docket Date': '',
 'Docket Type': '',
 'ANS Filed': '',
 'CR Number': '',
 'Plaintiff Name': ['ATTORNEY GENERAL', 'MARTINEZ, ROSALINDA'],
 'Plaintiff Attorney': ['ROLANDO MENCHACA'],
 'Defendant Name': ['MARTINEZ, HECTOR RENE'],
 'Defendant Attorney': []}

# Automate Process

Now that I can gather all available info for any single case on a page, build a function that can retrieve it all for all cases on all pages. Then convert the data to a dataframe and return it.

In [140]:
def extract_pdf_data(text):
    """
    This function takes in a page of text from a pdf file. It will extract all available information for each case on
    the page, put it all into dictionaries, and then return a list containing the other dictionaries for each
    case.
    
    Parameter:
        - text: The text of a single pdf page.
        
    Returns:
        - cases: A list containing the dictionaries of all cases.
    """
    
    #Set up the container list
    case_dicts = []
    
    #First, strip leading and trailing whitespaces
    text = text.strip()
    
    #Separate header and body text data
    #For header
    header = text[:392]
    
    #For body
    text = text[392:]
    
    #Remove headers from all subsequent pages
    text = re.sub(r"""COUNTY OF MAVERICK 293RD DISTRICT COURT CIVIL PENDING CASES AS OF .{10}

                                           RAN ON .{20,26} PAGE: \d{1,3}

        CAUSE #             FILE DATE CAUSE OF ACTION           DOCKT DATE  DOCKET TYPE       ANS FILED  CR CASE #

PLAINTIFF NAME                PLAINTIFF ATTORNEY        DEFENDANT NAME                DEFENDANT ATTORNEY""", '', text)
    
    #Use if statement to check for county names inside the header info
    if header.count('MAVERICK') >= 1:
        county = 'Maverick'
    elif header.count('DIMMIT') >= 1:
        county = 'Dimmit'
    elif header.count('ZAVALA') >= 1:
        county = 'Zavala'
    else:
        county = 'Something went wrong!'
    
    #Now split each case by the dashed lines
    text = text.split('-------------------------------------------------------------------------------------------------------------')
    
    #For testing
    cases = []
    #For testing
    for case in text:
        cases.extend(case.split('\x0c'))
        
    #For testing
    #Drop whitespace entries
    cases = list(filter(is_whitespace, cases))
    
    #Drop the last case. It only consists of the count of cases in the pdf. Not needed here
    cases.pop()
    
    #How many cases?
    print(f"Collecting Data From {len(cases)} Cases...")
    
    #Loop through each case and build a dictionary with its info
    for case_num, case in enumerate(cases):
        
        #Initialize the temp dict
        temp_dict = {}
        
        #Strip the leading whitesapces only
        case = case.lstrip()
        
        #Split on the new lines. 
        case_info = case.split('\n')
        
        #Get the cause number
        cause_num = case_info[0][:19].strip()

        #Get the file date
        file_date = case_info[0][15:30].strip()

        #Get the Cause of Action
        coa = case_info[0][30:56].strip()

        #Get the docket_date
        docket_date = case_info[0][56:68].strip()

        #Get the docket type
        docket_type = case_info[0][68:86].strip()

        #Get the ANS File
        ans_date = case_info[0][86:].strip()

        #Get the CR Number
        cr_num = case_info[0][96:].strip()
        
        #Get the names associated with each label
        #Will need to set up the lists before the loop. They will be added to the dict after the loop
        plaintiff_names = []
        plaintiff_attorneys = []
        defendant_names = []
        defendant_attorneys = []

        #The names always start on the third line (index = 2)
        for i in range(2, len(case_info)):
            #Get the plaintiff name on current line
            plaintiff_name = case_info[i][:38]

            #Check if plaintiff_name is all whitesapace. If not, strip it and add to names list
            #Also check that the string is not empty
            if plaintiff_name.isspace() == False and len(plaintiff_name) > 0:
                plaintiff_names.append(plaintiff_name.strip())

            #Get the plaintiff attorney on current line
            plaintiff_attorney = case_info[i][38:64]

            #Check if plaintiff_attorney is all whitespace. If not, strip it and add to list
            #Also check that the string is not empty
            if plaintiff_attorney.isspace() == False and len(plaintiff_attorney) > 0:
                plaintiff_attorneys.append(plaintiff_attorney.strip())

            #Get the defendant name on current line
            defendant_name = case_info[i][64:94]

            #Check if defendant_name is all whitespace. If not, strip it and add to list
            #Also check that the string is not empty
            if defendant_name.isspace() == False and len(defendant_name) > 0:
                defendant_names.append(defendant_name.strip())

            #Get the defendant attorney on current line
            defendant_attorney = case_info[i][94:]

            #Check if the defendant_attorney is all whitespace. If not, strip it and add to list
            #Also check that the string is not empty
            if defendant_attorney.isspace() == False and len(defendant_attorney) > 0:
                defendant_attorneys.append(defendant_attorney.strip())
              
        
        #Now put all the info into a temp dict. Check that each entry actually exists though
        #If the entry does not exist, just enter None
        temp_dict['County'] = county
        temp_dict['Cause Number'] = cause_num
        temp_dict['File Date'] = file_date
        temp_dict['Cause of Action'] = coa
        temp_dict['Docket Date'] = docket_date
        temp_dict['Docket Type'] = docket_type
        temp_dict['ANS File'] = ans_date
        temp_dict['CR Number'] = cr_num
        temp_dict['Plaintiff Name'] = plaintiff_names
        temp_dict['Plaintiff Attorney'] = plaintiff_attorneys
        temp_dict['Defendant Name'] = defendant_names
        temp_dict['Defendant Attorney'] = defendant_attorneys
        
        #Finally, append this dictionary to the container dict
        case_dicts.append(temp_dict)
        
    return case_dicts
        

In [56]:
def build_cases_dataframe(pdf_path):
    """
    This function reads in a pdf and extracts all available information for each case. It then returns a dataframe.
    
    Parameter:
        - pdf_path: The file path for the pdf to be read.
        
    Returns:
        - df: A dataframe of the resulting case information
    """
    
    #Set up the container list
    container_list = []
    
    #Set up resource manager to handle pdf content. text, images, etc.
    resource_manager = PDFResourceManager()

    #Used to display text
    fake_file_handle = io.StringIO()

    #Set up converter
    converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())

    #Set up page interpreter
    page_interpreter = PDFPageInterpreter(resource_manager, converter)

    with open(pdf_path, 'rb') as fh:

        for page in PDFPage.get_pages(fh, caching=True, check_extractable=True):
            #Process the current page
            page_interpreter.process_page(page)
            
        #Save the current page's text to a variable
        text = fake_file_handle.getvalue()

        #Extract info from current page
        case_dicts = extract_pdf_data(text)

        #Extend case_dicts to container_dict
        container_list.extend(case_dicts)

    # close open handles
    converter.close()
    fake_file_handle.close()
    
    #Build dataframe
    df = pd.DataFrame(container_list)
    
    return df

In [57]:
#For testing
df = build_cases_dataframe('/Users/johnathonsmith/Downloads/293RDCV.PEND.CASE_1642601189564_-1416322429.pdf')

In [58]:
df

,County,Cause Number,File Date,Cause of Action,Docket Date,Docket Type,ANS File,CR Number,Plaintiff Name,Plaintiff Attorney,Defendant Name,Defendant Attorney
0,Maverick,7815-OTH,02/22/1985,1,,,,,"[ATTORNEY GENERAL, MENDEZ, OLGA LIDIA]",[MARIA LARSON],"[MENDEZ, CARLOS GILBERTO]",[]
1,Maverick,90-11-10256-CV,11/08/1990,1,,,,,"[ATTORNEY GENERAL, MARTINEZ, ROSALINDA]",[ROLANDO MENCHACA],"[MARTINEZ, HECTOR RENE]",[]
2,Maverick,90-12-10326-CV,12/14/1990,INJURY OR DAMAGE OTHER T,,,,,"[DIAZ CARPENTER, ENRIQUETA]",[JAY T. KIMBROUGH],"[BOWLES JR., RUDY]",[RICHARD TINSMAN]
3,Maverick,91-06-10598-CV,10/22/1993,"MOTION ENFORCEMENT, TO M",09/13/1994,MOTION TO ENFORC,,,"[ATTORNEY GENERAL, FUENTES, MARIA EUGENIA]",[ROLANDO MENCHACA],"[ARROYO, JUAN ANTONIO]",[]
4,Maverick,95-01-12880-CV,01/05/1995,1,,,,,"[ATTORNEY GENERAL, LARA, MARIA GUADALUPE]",[ENRIQUE FERNANDEZ],"[LARA, HOMERO]",[]
...,...,...,...,...,...,...,...,...,...,...,...,...
1168,Maverick,22-01-40738-MCV,01/14/2022,DIVORCE - CHILDREN,,,,,[ABOYTES CYNTHIA],[JAD P HARPER],[ABOYTES ANDRES],[]
1169,Maverick,22-01-40740-MCV,01/14/2022,DEBT/CONTRACT - DEBT COL,,,,,[SYNCHRONY BANK],[ZHANG JESSICA],[CASAS EMILIO],[]
1170,Maverick,22-01-40742-MCV,01/14/2022,INJURY OR DAMAGE INVOLVI,,,,,"[CASARES JR CARLOS, CASARES EDUARDO]","[VICTOR E. PERRY, VICTOR E. PERRY]",[FUENTES DAVID],[]
1171,Maverick,22-01-40744-MCV,01/17/2022,DEBT/CONTRACT - DEBT COL,,,,,"[CROWN ASSET MANAGEMENT, LLC]",[BROGAN COLIN PATRICK],[BALDERAS PAULA J],[]


# Something Went Wrong

Just discovered that the PDF pages are not independent. As in, some case information can bleed over into the next page. So the process I used to extract the information won't work as is. I'll work on it some more tomorrow.

__Begin Testing__

In [59]:
#Set up resource manager to handle pdf content. text, images, etc.
resource_manager = PDFResourceManager()

#Used to display text
fake_file_handle = io.StringIO()

#Set up converter
converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())

#Set up page interpreter
page_interpreter = PDFPageInterpreter(resource_manager, converter)

with open('/Users/johnathonsmith/Downloads/293RDCV.PEND.CASE_1642601189564_-1416322429.pdf', 'rb') as fh:

    for page_number, page in enumerate(PDFPage.get_pages(fh,
                                  caching=True,
                                  maxpages = 14,
                                  check_extractable=True)):
        #Only read page 1 for now (index = 0)
        if page_number == 0:
            page_interpreter.process_page(page)

    text = fake_file_handle.getvalue()

# close open handles
converter.close()
fake_file_handle.close()

print(text)

                COUNTY OF MAVERICK 293RD DISTRICT COURT CIVIL PENDING CASES AS OF 01/19/2022

                                           RAN ON JANUARY 19, 2022 09:02am PAGE: 1

        CAUSE #             FILE DATE CAUSE OF ACTION           DOCKT DATE  DOCKET TYPE       ANS FILED  CR CASE #

PLAINTIFF NAME                PLAINTIFF ATTORNEY        DEFENDANT NAME                DEFENDANT ATTORNEY

        7815-OTH           02/22/1985 1                                                                  

        ATTORNEY GENERAL              MARIA LARSON              MENDEZ, CARLOS GILBERTO                            
        MENDEZ, OLGA LIDIA                                                                                         
        -------------------------------------------------------------------------------------------------------------
        90-11-10256-CV     11/08/1990 1                                                                  

        ATTORNEY GENERAL            

In [60]:
#Begin testing function
my_list = extract_pdf_data(text)

In [61]:
my_list

[{'County': 'Maverick',
  'Cause Number': '7815-OTH',
  'File Date': '02/22/1985',
  'Cause of Action': '1',
  'Docket Date': '',
  'Docket Type': '',
  'ANS File': '',
  'CR Number': '',
  'Plaintiff Name': ['ATTORNEY GENERAL', 'MENDEZ, OLGA LIDIA'],
  'Plaintiff Attorney': ['MARIA LARSON'],
  'Defendant Name': ['MENDEZ, CARLOS GILBERTO'],
  'Defendant Attorney': []},
 {'County': 'Maverick',
  'Cause Number': '90-11-10256-CV',
  'File Date': '11/08/1990',
  'Cause of Action': '1',
  'Docket Date': '',
  'Docket Type': '',
  'ANS File': '',
  'CR Number': '',
  'Plaintiff Name': ['ATTORNEY GENERAL', 'MARTINEZ, ROSALINDA'],
  'Plaintiff Attorney': ['ROLANDO MENCHACA'],
  'Defendant Name': ['MARTINEZ, HECTOR RENE'],
  'Defendant Attorney': []},
 {'County': 'Maverick',
  'Cause Number': '90-12-10326-CV',
  'File Date': '12/14/1990',
  'Cause of Action': 'INJURY OR DAMAGE OTHER T',
  'Docket Date': '',
  'Docket Type': '',
  'ANS File': '',
  'CR Number': '',
  'Plaintiff Name': ['DIAZ CAR

In [62]:
pd.DataFrame(my_list)

,County,Cause Number,File Date,Cause of Action,Docket Date,Docket Type,ANS File,CR Number,Plaintiff Name,Plaintiff Attorney,Defendant Name,Defendant Attorney
0,Maverick,7815-OTH,02/22/1985,1,,,,,"[ATTORNEY GENERAL, MENDEZ, OLGA LIDIA]",[MARIA LARSON],"[MENDEZ, CARLOS GILBERTO]",[]
1,Maverick,90-11-10256-CV,11/08/1990,1,,,,,"[ATTORNEY GENERAL, MARTINEZ, ROSALINDA]",[ROLANDO MENCHACA],"[MARTINEZ, HECTOR RENE]",[]
2,Maverick,90-12-10326-CV,12/14/1990,INJURY OR DAMAGE OTHER T,,,,,"[DIAZ CARPENTER, ENRIQUETA]",[JAY T. KIMBROUGH],"[BOWLES JR., RUDY]",[RICHARD TINSMAN]
3,Maverick,91-06-10598-CV,10/22/1993,"MOTION ENFORCEMENT, TO M",09/13/1994,MOTION TO ENFORC,,,"[ATTORNEY GENERAL, FUENTES, MARIA EUGENIA]",[ROLANDO MENCHACA],"[ARROYO, JUAN ANTONIO]",[]
4,Maverick,95-01-12880-CV,01/05/1995,1,,,,,"[ATTORNEY GENERAL, LARA, MARIA GUADALUPE]",[ENRIQUE FERNANDEZ],"[LARA, HOMERO]",[]
5,Maverick,96-05-13796-CV,05/08/1996,1,,,,,"[ATTORNEY GENERAL, HERNANDEZ, OLGA E.]",[POLLY J BONE],"[HERNANDEZ, JUAN F.]",[]
6,Maverick,96-12-14214-CV,12/20/1996,OTHER CIVIL CASES,02/18/1998,WON JURY TRIAL,,,[THE STATE OF TEXAS],[MELISSA DAYLONG],[APPROX. 440 POUNDS OF MARIHU],[]


# Success!

Now I will fix the multiple pages issue.

In [63]:
#Load two pages
#Set up resource manager to handle pdf content. text, images, etc.
resource_manager = PDFResourceManager()

#Used to display text
fake_file_handle = io.StringIO()

#Set up converter
converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())

#Set up page interpreter
page_interpreter = PDFPageInterpreter(resource_manager, converter)

with open('/Users/johnathonsmith/Downloads/293RDCV.PEND.CASE_1642601189564_-1416322429.pdf', 'rb') as fh:

    for page_number, page in enumerate(PDFPage.get_pages(fh,
                                  caching=True,
                                  maxpages = 2,
                                  check_extractable=True)):
        #Only read page 1 for now (index = 0)
        #if page_number == 0:
        page_interpreter.process_page(page)

    text = fake_file_handle.getvalue()

# close open handles
converter.close()
fake_file_handle.close()

print(text)

                COUNTY OF MAVERICK 293RD DISTRICT COURT CIVIL PENDING CASES AS OF 01/19/2022

                                           RAN ON JANUARY 19, 2022 09:02am PAGE: 1

        CAUSE #             FILE DATE CAUSE OF ACTION           DOCKT DATE  DOCKET TYPE       ANS FILED  CR CASE #

PLAINTIFF NAME                PLAINTIFF ATTORNEY        DEFENDANT NAME                DEFENDANT ATTORNEY

        7815-OTH           02/22/1985 1                                                                  

        ATTORNEY GENERAL              MARIA LARSON              MENDEZ, CARLOS GILBERTO                            
        MENDEZ, OLGA LIDIA                                                                                         
        -------------------------------------------------------------------------------------------------------------
        90-11-10256-CV     11/08/1990 1                                                                  

        ATTORNEY GENERAL            

In [64]:
#Try using regex to remove the headers on all pages
import re

In [65]:
re.sub(r"""COUNTY OF MAVERICK 293RD DISTRICT COURT CIVIL PENDING CASES AS OF 01/19/2022

                                           RAN ON JANUARY 19, 2022 09:02am PAGE: \d{1,3}

        CAUSE #             FILE DATE CAUSE OF ACTION           DOCKT DATE  DOCKET TYPE       ANS FILED  CR CASE #

PLAINTIFF NAME                PLAINTIFF ATTORNEY        DEFENDANT NAME                DEFENDANT ATTORNEY""", '', text)

'                \n\n        7815-OTH           02/22/1985 1                                                                  \n\n        ATTORNEY GENERAL              MARIA LARSON              MENDEZ, CARLOS GILBERTO                            \n        MENDEZ, OLGA LIDIA                                                                                         \n        -------------------------------------------------------------------------------------------------------------\n        90-11-10256-CV     11/08/1990 1                                                                  \n\n        ATTORNEY GENERAL              ROLANDO MENCHACA          MARTINEZ, HECTOR RENE                              \n        MARTINEZ, ROSALINDA                                                                                        \n        -------------------------------------------------------------------------------------------------------------\n        90-12-10326-CV     12/14/1990 INJURY OR DAMAGE 

In [66]:
#Test function on two pages
#First reload the two pages
#Load two pages
#Set up resource manager to handle pdf content. text, images, etc.
resource_manager = PDFResourceManager()

#Used to display text
fake_file_handle = io.StringIO()

#Set up converter
converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())

#Set up page interpreter
page_interpreter = PDFPageInterpreter(resource_manager, converter)

with open('/Users/johnathonsmith/Downloads/293RDCV.PEND.CASE_1642601189564_-1416322429.pdf', 'rb') as fh:

    for page_number, page in enumerate(PDFPage.get_pages(fh,
                                  caching=True,
                                  maxpages = 3,
                                  check_extractable=True)):
        #Only read page 1 for now (index = 0)
        #if page_number == 0:
        page_interpreter.process_page(page)

    text = fake_file_handle.getvalue()

# close open handles
converter.close()
fake_file_handle.close()

print(text)

                COUNTY OF MAVERICK 293RD DISTRICT COURT CIVIL PENDING CASES AS OF 01/19/2022

                                           RAN ON JANUARY 19, 2022 09:02am PAGE: 1

        CAUSE #             FILE DATE CAUSE OF ACTION           DOCKT DATE  DOCKET TYPE       ANS FILED  CR CASE #

PLAINTIFF NAME                PLAINTIFF ATTORNEY        DEFENDANT NAME                DEFENDANT ATTORNEY

        7815-OTH           02/22/1985 1                                                                  

        ATTORNEY GENERAL              MARIA LARSON              MENDEZ, CARLOS GILBERTO                            
        MENDEZ, OLGA LIDIA                                                                                         
        -------------------------------------------------------------------------------------------------------------
        90-11-10256-CV     11/08/1990 1                                                                  

        ATTORNEY GENERAL            

In [67]:
#Now call the function
my_list = extract_pdf_data(text)

In [68]:
my_list

[{'County': 'Maverick',
  'Cause Number': '7815-OTH',
  'File Date': '02/22/1985',
  'Cause of Action': '1',
  'Docket Date': '',
  'Docket Type': '',
  'ANS File': '',
  'CR Number': '',
  'Plaintiff Name': ['ATTORNEY GENERAL', 'MENDEZ, OLGA LIDIA'],
  'Plaintiff Attorney': ['MARIA LARSON'],
  'Defendant Name': ['MENDEZ, CARLOS GILBERTO'],
  'Defendant Attorney': []},
 {'County': 'Maverick',
  'Cause Number': '90-11-10256-CV',
  'File Date': '11/08/1990',
  'Cause of Action': '1',
  'Docket Date': '',
  'Docket Type': '',
  'ANS File': '',
  'CR Number': '',
  'Plaintiff Name': ['ATTORNEY GENERAL', 'MARTINEZ, ROSALINDA'],
  'Plaintiff Attorney': ['ROLANDO MENCHACA'],
  'Defendant Name': ['MARTINEZ, HECTOR RENE'],
  'Defendant Attorney': []},
 {'County': 'Maverick',
  'Cause Number': '90-12-10326-CV',
  'File Date': '12/14/1990',
  'Cause of Action': 'INJURY OR DAMAGE OTHER T',
  'Docket Date': '',
  'Docket Type': '',
  'ANS File': '',
  'CR Number': '',
  'Plaintiff Name': ['DIAZ CAR

# Success!

Now try extracting the whole document and creating a dataframe.

In [69]:
df = build_cases_dataframe('/Users/johnathonsmith/Downloads/293RDCV.PEND.CASE_1642601189564_-1416322429.pdf')

In [70]:
df

,County,Cause Number,File Date,Cause of Action,Docket Date,Docket Type,ANS File,CR Number,Plaintiff Name,Plaintiff Attorney,Defendant Name,Defendant Attorney
0,Maverick,7815-OTH,02/22/1985,1,,,,,"[ATTORNEY GENERAL, MENDEZ, OLGA LIDIA]",[MARIA LARSON],"[MENDEZ, CARLOS GILBERTO]",[]
1,Maverick,90-11-10256-CV,11/08/1990,1,,,,,"[ATTORNEY GENERAL, MARTINEZ, ROSALINDA]",[ROLANDO MENCHACA],"[MARTINEZ, HECTOR RENE]",[]
2,Maverick,90-12-10326-CV,12/14/1990,INJURY OR DAMAGE OTHER T,,,,,"[DIAZ CARPENTER, ENRIQUETA]",[JAY T. KIMBROUGH],"[BOWLES JR., RUDY]",[RICHARD TINSMAN]
3,Maverick,91-06-10598-CV,10/22/1993,"MOTION ENFORCEMENT, TO M",09/13/1994,MOTION TO ENFORC,,,"[ATTORNEY GENERAL, FUENTES, MARIA EUGENIA]",[ROLANDO MENCHACA],"[ARROYO, JUAN ANTONIO]",[]
4,Maverick,95-01-12880-CV,01/05/1995,1,,,,,"[ATTORNEY GENERAL, LARA, MARIA GUADALUPE]",[ENRIQUE FERNANDEZ],"[LARA, HOMERO]",[]
...,...,...,...,...,...,...,...,...,...,...,...,...
1168,Maverick,22-01-40738-MCV,01/14/2022,DIVORCE - CHILDREN,,,,,[ABOYTES CYNTHIA],[JAD P HARPER],[ABOYTES ANDRES],[]
1169,Maverick,22-01-40740-MCV,01/14/2022,DEBT/CONTRACT - DEBT COL,,,,,[SYNCHRONY BANK],[ZHANG JESSICA],[CASAS EMILIO],[]
1170,Maverick,22-01-40742-MCV,01/14/2022,INJURY OR DAMAGE INVOLVI,,,,,"[CASARES JR CARLOS, CASARES EDUARDO]","[VICTOR E. PERRY, VICTOR E. PERRY]",[FUENTES DAVID],[]
1171,Maverick,22-01-40744-MCV,01/17/2022,DEBT/CONTRACT - DEBT COL,,,,,"[CROWN ASSET MANAGEMENT, LLC]",[BROGAN COLIN PATRICK],[BALDERAS PAULA J],[]


# Nearly a Success! 

It looks like my function failed to pick up 6 out of the 1,179 total cases. I need to figure that out next. After looking through the original PDF, I believe I have found the problem. Sometimes, when a case ends, and a new page begins, the PDF fails to generate the dashed line. I used the dashed line to separate the cases, so without it, some of them have been put together.

In [133]:
#Load an example of the above problem. I found it on pages 21 and 22 of the PDF
#Load two pages
#Set up resource manager to handle pdf content. text, images, etc.
resource_manager = PDFResourceManager()

#Used to display text
fake_file_handle = io.StringIO()

#Set up converter
converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())

#Set up page interpreter
page_interpreter = PDFPageInterpreter(resource_manager, converter)

with open('/Users/johnathonsmith/Downloads/293RDCV.PEND.CASE_1642601189564_-1416322429.pdf', 'rb') as fh:

    for page_number, page in enumerate(PDFPage.get_pages(fh,
                                  caching=True,
                                  maxpages = 23,
                                  check_extractable=True)):
        #Only read page 21 and 22 for now (index = 20, 21)
        if page_number == 20 or page_number == 21:
            page_interpreter.process_page(page)

    text = fake_file_handle.getvalue()

# close open handles
converter.close()
fake_file_handle.close()

print(text)

                COUNTY OF MAVERICK 293RD DISTRICT COURT CIVIL PENDING CASES AS OF 01/19/2022

                                           RAN ON JANUARY 19, 2022 09:02am PAGE: 21

        CAUSE #             FILE DATE CAUSE OF ACTION           DOCKT DATE  DOCKET TYPE       ANS FILED  CR CASE #

PLAINTIFF NAME                PLAINTIFF ATTORNEY        DEFENDANT NAME                DEFENDANT ATTORNEY

        15-01-30572-MCV    12/16/2019 DIVORCE - NO CHILDREN     09/10/2021  STATUS HEARING               

        VALDEZ, ALEJANDRA SOFIA       PRISCILLA PUENTE-CHACON   ROCHA, FERNANDO ANTONIO       FRANCISCO MARTINEZ II
        -------------------------------------------------------------------------------------------------------------
        15-01-30610-MCV    01/21/2015 2                         11/09/2015  STATUS REPORT                

        ESPINOZA, MANUEL              WES HOLLAND               ALLSTATE VEHICLE AND PROPERT  ROGER D. HIGGINS     
        ESPINOZA, MARINA           

In [134]:
#Remove the headers using regex
text = re.sub(r"""COUNTY OF MAVERICK 293RD DISTRICT COURT CIVIL PENDING CASES AS OF .{10}

                                           RAN ON .{20,26} PAGE: \d{1,3}

        CAUSE #             FILE DATE CAUSE OF ACTION           DOCKT DATE  DOCKET TYPE       ANS FILED  CR CASE #

PLAINTIFF NAME                PLAINTIFF ATTORNEY        DEFENDANT NAME                DEFENDANT ATTORNEY""", '', text)

In [135]:
text

'                \n\n        15-01-30572-MCV    12/16/2019 DIVORCE - NO CHILDREN     09/10/2021  STATUS HEARING               \n\n        VALDEZ, ALEJANDRA SOFIA       PRISCILLA PUENTE-CHACON   ROCHA, FERNANDO ANTONIO       FRANCISCO MARTINEZ II\n        -------------------------------------------------------------------------------------------------------------\n        15-01-30610-MCV    01/21/2015 2                         11/09/2015  STATUS REPORT                \n\n        ESPINOZA, MANUEL              WES HOLLAND               ALLSTATE VEHICLE AND PROPERT  ROGER D. HIGGINS     \n        ESPINOZA, MARINA                                        STEWART JR., ALLAN                                 \n        -------------------------------------------------------------------------------------------------------------\n        15-01-30612-MCV    01/21/2015 DIVORCE - CHILDREN        01/07/2022  ENTRY OF ORDER               \n                           10/30/2019\n\n         GARZA, DIANA   

In [136]:
#Now split on the dashed lines
text = text.split('-------------------------------------------------------------------------------------------------------------')

In [137]:
#How many splits were generated?
len(text)

15

In [138]:
text[7]

'\n        15-02-30752-MCV    02/11/2015 OTHER CIVIL CASES         06/08/2015  ENTRY OF JUDGMEN             \n\n        MONSA, LTD                    BLAYNE FISHER             DOBBINS, STEVE                ELIZABETH SANDOVAL CA\n                                                                STATE FARM LLOYDS                                  \n\n\x0c                \n\n        15-02-30758-MCV    02/12/2015 2                         06/08/2015  PRE-TRIAL & DOCK             \n\n        MONSA, LTD                    BLAYNE FISHER             RASMUSSEN, CURTIS             ELIZABETH SANDOVAL CA\n                                                                STATE FARM LLOYDS                                  \n        '

In [139]:
def is_whitespace(x):
    if x.isspace() == True:
        return False
    else:
        return True

In [141]:
df = build_cases_dataframe('/Users/johnathonsmith/Downloads/293RDCV.PEND.CASE_1642601189564_-1416322429.pdf')

# Getting there

Function still needs work